<a href="https://colab.research.google.com/github/Gionauta/Avanti-Bootcamp-Machine-Learning/blob/main/YOLO_FACE_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📦 Required Dependencies

In [ ]:
!pip install pyproject-toml

In [ ]:
!pip install super-gradients

In [ ]:

import os
import shutil
import random
from tqdm.notebook import tqdm
from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train, coco_detection_yolo_format_val
import torch
from super_gradients.training import models
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback


# 💾 Datasets and DataLoaders

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset_params = {
    'data_dir':'/content/drive/MyDrive/Avanti Bootcamp ML/face-detection-dataset',
    'train_images_dir':'/content/drive/MyDrive/Avanti Bootcamp ML/face-detection-dataset/images/train',
    'train_labels_dir':'/content/drive/MyDrive/Avanti Bootcamp ML/face-detection-dataset/labels/train',
    'val_images_dir':'/content/drive/MyDrive/Avanti Bootcamp ML/face-detection-dataset/images/val',
    'val_labels_dir':'/content/drive/MyDrive/Avanti Bootcamp ML/face-detection-dataset/labels/val',
    'test_images_dir':'/content/drive/MyDrive/Avanti Bootcamp ML/face-detection-dataset/images/train',
    'test_labels_dir':'/content/drive/MyDrive/Avanti Bootcamp ML/face-detection-dataset/labels/train',
    'classes': ['face']
}


In [ ]:
MODEL_ARCH = 'yolo_nas_l'
DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 8 #pode ser alterado (aumentar valor)
MAX_EPOCHS = 20 #pode ser alterado (aumentar valor)
CHECKPOINT_DIR = f'/content/drive/MyDrive/Avanti Bootcamp ML/face-detection-dataset/Best_Model/yolo_nas_face'
EXPERIMENT_NAME = f'RUN_20231117_151210_044804'


In [ ]:
train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 1
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 1
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 1
    }
)

In [ ]:
train_data.dataset.transforms

In [ ]:
train_data.dataset.plot()

# ⚙️ Initializing the model

In [ ]:
model = models.get(
    MODEL_ARCH,
    num_classes=len(dataset_params['classes']),
    pretrained_weights="coco"
)

[2023-11-17 15:12:08] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.


# 🔍 Training parameters

In [ ]:
train_params = {
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step", #pode ser alterado (LINEAR_STEP = "LinearEpochLRWarmup"LINEAR_EPOCH_STEP = "LinearEpochLRWarmup" LINEAR_BATCH_STEP = "LinearBatchLRWarmup")
    "warmup_initial_lr": 1e-6, #pode ser alterado (float)
    "lr_warmup_epochs": 3,#pode ser alterado (float)
    "initial_lr": 5e-4, #pode ser alterado (float)
    "lr_mode": "cosine", #pode ser alterado ('StepLRScheduler','PolyLRScheduler','CosineLRScheduler','FunctionLRScheduler')
    "cosine_final_lr_ratio": 0.1, #pode ser alterado ( dev ser levado em conta o lr_mode escolhido)
    "optimizer": "Adam", #pode ser alterado (SGD = "SGD"ADAM = "Adam" ADAMW = "AdamW" RMS_PROP = "RMSprop"RMS_PROP_TF = "RMSpropTF" LAMB = "Lamb" LION = "Lion")
    "optimizer_params": {"weight_decay": 0.0001},#pode ser alterado (  :param weight_decay: value to set for the non BN and bias parameter(float, optional))
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},#param decay: The maximum decay value
    #:param step: Current training step. The unit-range training percentage can be obtained by `step / total_steps`.
    #:param total_steps:  Total number of training steps.
    #constant: "ema_params": {"decay": 0.9999, "decay_type": "constant"} threshold: "ema_params": {"decay": 0.9999, "decay_type": "threshold"}
    #exp: "ema_params": {"decay": 0.9999, "decay_type": "exp", "beta": 15}
    "max_epochs": MAX_EPOCHS,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}


# ⏳ Training the model

In [ ]:
from super_gradients.training import Trainer

trainer = Trainer(experiment_name=EXPERIMENT_NAME, ckpt_root_dir=CHECKPOINT_DIR)

In [ ]:
trainer.train(model=model,
              training_params=train_params,
              train_loader=train_data,
              valid_loader=val_data)

# 📈 Inference and Visualization

In [ ]:
best_model = models.get(
    MODEL_ARCH,
    num_classes=len(dataset_params['classes']),
    checkpoint_path=f"{CHECKPOINT_DIR}/{EXPERIMENT_NAME}/average_model.pth"
).to(DEVICE)

In [ ]:
img_1 = "/content/drive/MyDrive/Avanti Bootcamp ML/face-detection-dataset/Beatles-on-Radio.jpeg"
best_model.predict(img_1).show()

In [ ]:
img_2 = "/content/drive/MyDrive/Avanti Bootcamp ML/face-detection-dataset/alexandre-durao-20.jpeg"
best_model.predict(img_2).show()